<a href="https://colab.research.google.com/github/yaswanthkillampalli/cyber-threat-prediction/blob/main/data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Necessary Modules**

In [1]:
import sqlite3
import pandas as pd

**Load dataset from the generated CS**

In [2]:
try:
    df = pd.read_csv('/content/synthetic_ecommerce_orders.csv')
except FileNotFoundError:
    print("Please run the data generation script first to create 'synthetic_ecommerce_orders.csv'")
    exit()

**Create SQLite connection (in-memory database)**

In [3]:
conn = sqlite3.connect(":memory:")

**Write dataframe to SQL table**

In [4]:
df.to_sql("orders", conn, index=False, if_exists="replace")

5500

**Remove duplicate order entries based on OrderID**

In [5]:
print("--- Prompt 1: Removing duplicate order entries ---")
# Create a new table with unique orders, keeping the first one found
query_1 = """
CREATE TABLE orders_cleaned AS
SELECT * FROM (
    SELECT *,
           ROW_NUMBER() OVER(PARTITION BY OrderID ORDER BY OrderDate, OrderTime) as rn
    FROM orders
)
WHERE rn = 1;
"""
conn.execute(query_1)
df_1 = pd.read_sql_query("SELECT COUNT(*) as original_rows FROM orders;", conn)
df_1_cleaned = pd.read_sql_query("SELECT COUNT(*) as cleaned_rows FROM orders_cleaned;", conn)
print("Original row count:", df_1.iloc[0,0])
print("Cleaned row count:", df_1_cleaned.iloc[0,0])
print("\n" + "="*50 + "\n")

--- Prompt 1: Removing duplicate order entries ---
Original row count: 5500
Cleaned row count: 4982




**Fill missing Customer Ratings with the average rating for the respective product category**

In [6]:
print("--- Prompt 2: Filling missing Customer Ratings with category average ---")
# Using a correlated subquery to fill NULLs
query_2 = """
UPDATE orders_cleaned
SET CustomerRating = (
    SELECT AVG(T2.CustomerRating)
    FROM orders_cleaned AS T2
    WHERE T2.ProductCategory = orders_cleaned.ProductCategory
)
WHERE CustomerRating IS NULL;
"""
conn.execute(query_2)
conn.commit()
df_2 = pd.read_sql_query('SELECT ProductCategory, CustomerRating FROM orders_cleaned WHERE OrderStatus = "Delivered" LIMIT 10;', conn)
print(df_2)
print("\n" + "="*50 + "\n")

--- Prompt 2: Filling missing Customer Ratings with category average ---
  ProductCategory  CustomerRating
0           Books             2.7
1            Toys             3.4
2           Books             3.8
3  Home & Kitchen             1.5
4     Electronics             4.9
5          Beauty             1.5
6            Toys             2.3
7     Electronics             4.3
8           Books             2.4
9           Books             3.3




**Standardize Payment Method values to a consistent case**

In [7]:
print("--- Prompt 3: Standardizing Payment Method values ---")
query_3 = """
UPDATE orders_cleaned
SET PaymentMethod = INITCAP(TRIM(PaymentMethod));
"""
# SQLite doesn't have INITCAP, so we use a combination of UPPER and LOWER
query_3_sqlite = """
UPDATE orders_cleaned
SET PaymentMethod = UPPER(SUBSTR(TRIM(PaymentMethod), 1, 1)) || LOWER(SUBSTR(TRIM(PaymentMethod), 2));
"""
conn.execute(query_3_sqlite)
conn.commit()
df_3 = pd.read_sql_query('SELECT DISTINCT PaymentMethod FROM orders_cleaned;', conn)
print(df_3)
print("\n" + "="*50 + "\n")

--- Prompt 3: Standardizing Payment Method values ---
   PaymentMethod
0    Credit card
1  Bank transfer
2           None
3     Debit card
4         Paypal
5      Gift card




**Create a new column 'OrderYearMonth' from the 'OrderDate' field**

In [8]:
print("--- Prompt 4: Creating an 'OrderYearMonth' column ---")
query_4 = """
ALTER TABLE orders_cleaned ADD COLUMN OrderYearMonth TEXT;
UPDATE orders_cleaned
SET OrderYearMonth = STRFTIME('%Y-%m', OrderDate);
"""
# Running commands one by one
conn.execute("ALTER TABLE orders_cleaned ADD COLUMN OrderYearMonth TEXT;")
conn.execute("UPDATE orders_cleaned SET OrderYearMonth = STRFTIME('%Y-%m', OrderDate);")
conn.commit()
df_4 = pd.read_sql_query('SELECT OrderDate, OrderYearMonth FROM orders_cleaned LIMIT 5;', conn)
print(df_4)
print("\n" + "="*50 + "\n")

--- Prompt 4: Creating an 'OrderYearMonth' column ---
    OrderDate OrderYearMonth
0  2023-09-01        2023-09
1  2025-07-22        2025-07
2  2025-07-28        2025-07
3  2025-06-01        2025-06
4  2024-06-02        2024-06




**Count the number of orders for each Product Category**

In [9]:
print("--- Prompt 5: Number of orders per Product Category ---")
query_5 = """
SELECT ProductCategory, COUNT(OrderID) AS num_orders
FROM orders_cleaned
GROUP BY ProductCategory
ORDER BY num_orders DESC;
"""
df_5 = pd.read_sql_query(query_5, conn)
print(df_5)
print("\n" + "="*50 + "\n")

--- Prompt 5: Number of orders per Product Category ---
  ProductCategory  num_orders
0            Toys         914
1     Electronics         912
2          Sports         870
3           Books         728
4  Home & Kitchen         566
5          Beauty         504
6        Clothing         488




**Calculate the total revenue generated by each Payment Method**

In [10]:
print("--- Prompt 6: Total revenue by Payment Method ---")
query_6 = """
SELECT PaymentMethod, SUM(TotalAmount) AS total_revenue
FROM orders_cleaned
WHERE OrderStatus = 'Delivered'
GROUP BY PaymentMethod
ORDER BY total_revenue DESC;
"""
df_6 = pd.read_sql_query(query_6, conn)
print(df_6)
print("\n" + "="*50 + "\n")

--- Prompt 6: Total revenue by Payment Method ---
   PaymentMethod  total_revenue
0    Credit card      220763.92
1         Paypal      202742.58
2      Gift card      119642.65
3     Debit card      117445.60
4  Bank transfer      112821.05




**Find the top 5 best-selling products by quantity sold**

In [11]:
print("--- Prompt 7: Top 5 best-selling products by quantity ---")
query_7 = """
SELECT ProductID, SUM(Quantity) AS total_quantity_sold
FROM orders_cleaned
WHERE OrderStatus IN ('Delivered', 'Shipped')
GROUP BY ProductID
ORDER BY total_quantity_sold DESC
LIMIT 5;
"""
df_7 = pd.read_sql_query(query_7, conn)
print(df_7)
print("\n" + "="*50 + "\n")

--- Prompt 7: Top 5 best-selling products by quantity ---
  ProductID  total_quantity_sold
0   PID1633                   61
1   PID1800                   56
2   PID9371                   54
3   PID8143                   53
4   PID5069                   53




**Compare the average customer rating for each shipping method**

In [12]:
print("--- Prompt 8: Average customer rating by shipping method ---")
query_8 = """
SELECT ShippingMethod, AVG(CustomerRating) AS avg_rating
FROM orders_cleaned
WHERE OrderStatus = 'Delivered'
GROUP BY ShippingMethod
ORDER BY avg_rating DESC;
"""
df_8 = pd.read_sql_query(query_8, conn)
print(df_8)
print("\n" + "="*50 + "\n")

--- Prompt 8: Average customer rating by shipping method ---
  ShippingMethod  avg_rating
0       Next-Day    2.972754
1       Standard    2.966771
2        Express    2.962154




**Identify the month with the highest number of cancelled orders**

In [13]:
print("--- Prompt 9: Month with the most cancelled orders ---")
query_9 = """
SELECT OrderYearMonth, COUNT(OrderID) AS cancelled_orders
FROM orders_cleaned
WHERE OrderStatus = 'Cancelled'
GROUP BY OrderYearMonth
ORDER BY cancelled_orders DESC
LIMIT 1;
"""
df_9 = pd.read_sql_query(query_9, conn)
print(df_9)
print("\n" + "="*50 + "\n")

--- Prompt 9: Month with the most cancelled orders ---
  OrderYearMonth  cancelled_orders
0        2024-02                53




**Calculate the average number of days between order date and shipping date**

In [14]:
print("--- Prompt 10: Average shipping time ---")
query_10 = """
SELECT AVG(JULIANDAY(ShippingDate) - JULIANDAY(OrderDate)) AS avg_days_to_ship
FROM orders_cleaned
WHERE ShippingDate IS NOT NULL;
"""
df_10 = pd.read_sql_query(query_10, conn)
print(df_10)
print("\n" + "="*50 + "\n")

--- Prompt 10: Average shipping time ---
   avg_days_to_ship
0          2.009734




**Find the most frequent reason for order cancellations**

In [15]:
print("--- Prompt 11: Most frequent order cancellation reason ---")
query_11 = """
SELECT CancellationReason, COUNT(*) AS reason_count
FROM orders_cleaned
WHERE CancellationReason IS NOT NULL
GROUP BY CancellationReason
ORDER BY reason_count DESC
LIMIT 1;
"""
df_11 = pd.read_sql_query(query_11, conn)
print(df_11)
print("\n" + "="*50 + "\n")

--- Prompt 11: Most frequent order cancellation reason ---
  CancellationReason  reason_count
0     Fraud detected           226




**Calculate the percentage of total orders that have been returned**

In [16]:
print("--- Prompt 12: Percentage of returned orders ---")
query_12 = """
SELECT
    (SUM(CASE WHEN OrderStatus = 'Returned' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS return_rate_percentage
FROM orders_cleaned;
"""
df_12 = pd.read_sql_query(query_12, conn)
print(df_12)
print("\n" + "="*50 + "\n")

--- Prompt 12: Percentage of returned orders ---
   return_rate_percentage
0               20.634283




**Compute the monthly revenue trend (total sales per month)**

In [17]:
print("--- Prompt 13: Monthly revenue trend ---")
query_13 = """
SELECT OrderYearMonth, SUM(TotalAmount) AS monthly_revenue
FROM orders_cleaned
WHERE OrderStatus = 'Delivered'
GROUP BY OrderYearMonth
ORDER BY OrderYearMonth;
"""
df_13 = pd.read_sql_query(query_13, conn)
print(df_13)
print("\n" + "="*50 + "\n")

--- Prompt 13: Monthly revenue trend ---
   OrderYearMonth  monthly_revenue
0         2023-08          8258.62
1         2023-09         33146.13
2         2023-10         43075.68
3         2023-11         25896.09
4         2023-12         40389.47
5         2024-01         34445.94
6         2024-02         25543.14
7         2024-03         27018.44
8         2024-04         33320.30
9         2024-05         32354.29
10        2024-06         34351.00
11        2024-07         22246.00
12        2024-08         22551.03
13        2024-09         38562.52
14        2024-10         39084.58
15        2024-11         39328.87
16        2024-12         41504.14
17        2025-01         30874.72
18        2025-02         26385.52
19        2025-03         29347.17
20        2025-04         26543.08
21        2025-05         42384.22
22        2025-06         23482.55
23        2025-07         25049.74
24        2025-08         28272.56




**Identify the top 5 customers with the highest number of orders**

In [18]:
print("--- Prompt 14: Top 5 customers by order count ---")
query_14 = """
SELECT CustomerID, COUNT(OrderID) AS order_count
FROM orders_cleaned
GROUP BY CustomerID
ORDER BY order_count DESC
LIMIT 5;
"""
df_14 = pd.read_sql_query(query_14, conn)
print(df_14)
print("\n" + "="*50 + "\n")

--- Prompt 14: Top 5 customers by order count ---
  CustomerID  order_count
0    CUS6902            5
1    CUS4519            5
2    CUS3907            5
3    CUS9734            4
4    CUS9067            4




**Find the product category with the highest average product price**

In [19]:
print("--- Prompt 15: Product category with the highest average price ---")
query_15 = """
SELECT ProductCategory, AVG(ProductPrice) AS avg_price
FROM orders_cleaned
GROUP BY ProductCategory
ORDER BY avg_price DESC
LIMIT 1;
"""
df_15 = pd.read_sql_query(query_15, conn)
print(df_15)
print("\n" + "="*50 + "\n")

--- Prompt 15: Product category with the highest average price ---
  ProductCategory   avg_price
0  Home & Kitchen  281.194134




**Determine the percentage of orders for each shipping method**

In [20]:
print("--- Prompt 16: Percentage of orders per shipping method ---")
query_16 = """
SELECT ShippingMethod, COUNT(OrderID) * 100.0 / (SELECT COUNT(*) FROM orders_cleaned) AS percentage
FROM orders_cleaned
GROUP BY ShippingMethod
ORDER BY percentage DESC;
"""
df_16 = pd.read_sql_query(query_16, conn)
print(df_16)
print("\n" + "="*50 + "\n")

--- Prompt 16: Percentage of orders per shipping method ---
  ShippingMethod  percentage
0       Next-Day   34.022481
1        Express   33.239663
2       Standard   32.737856




**Calculate the average order value (AOV) for each product category**

In [21]:
print("--- Prompt 17: Average order value by category ---")
query_17 = """
SELECT ProductCategory, AVG(TotalAmount) as average_order_value
FROM orders_cleaned
GROUP BY ProductCategory
ORDER BY average_order_value DESC;
"""
df_17 = pd.read_sql_query(query_17, conn)
print(df_17)
print("\n" + "="*50 + "\n")

--- Prompt 17: Average order value by category ---
  ProductCategory  average_order_value
0  Home & Kitchen           870.636961
1     Electronics           832.348224
2            Toys           823.756105
3           Books           787.467225
4        Clothing           785.042746
5          Beauty           749.898155
6          Sports           702.560115




**List the top 3 customers who have returned the most orders**

In [22]:
print("--- Prompt 18: Top 3 customers by returned orders ---")
query_18 = """
SELECT CustomerID, COUNT(OrderID) as returned_orders
FROM orders_cleaned
WHERE OrderStatus = 'Returned'
GROUP BY CustomerID
ORDER BY returned_orders DESC
LIMIT 3;
"""
df_18 = pd.read_sql_query(query_18, conn)
print(df_18)
print("\n" + "="*50 + "\n")

--- Prompt 18: Top 3 customers by returned orders ---
  CustomerID  returned_orders
0    CUS9670                3
1    CUS4525                3
2    CUS1982                3




**Find the total number of items sold for each order status**

In [23]:
print("--- Prompt 19: Total items sold by order status ---")
query_19 = """
SELECT OrderStatus, SUM(Quantity) as total_items
FROM orders_cleaned
GROUP BY OrderStatus
ORDER BY total_items DESC;
"""
df_19 = pd.read_sql_query(query_19, conn)
print(df_19)
print("\n" + "="*50 + "\n")

--- Prompt 19: Total items sold by order status ---
  OrderStatus  total_items
0    Returned         3065
1  Processing         2988
2     Shipped         2973
3   Cancelled         2929
4   Delivered         2926




**Calculate the customer retention rate (simplified as customers who ordered more than once)**

In [24]:
print("--- Prompt 20: Percentage of repeat customers ---")
query_20 = """
WITH CustomerOrderCounts AS (
    SELECT CustomerID, COUNT(OrderID) as order_count
    FROM orders_cleaned
    GROUP BY CustomerID
)
SELECT
    SUM(CASE WHEN order_count > 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(CustomerID) AS repeat_customer_percentage
FROM CustomerOrderCounts;
"""
df_20 = pd.read_sql_query(query_20, conn)
print(df_20)
print("\n" + "="*50 + "\n")

--- Prompt 20: Percentage of repeat customers ---
   repeat_customer_percentage
0                   25.511274




**Close the connection**

In [ ]:
conn.close()